In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/Project 0/Project 0/Notebooks"
import sys
sys.path.append(path)
from funcs import *
from keras.preprocessing.image import array_to_img

# Visualizing AP and Control Predictions

In [ ]:
HybridDf = pd.read_csv(path+'/AP_Hybrid_Results.csv')
_ , _, test = getTrainTest(path,positiveClass='AP', valSet=True,imageFolder='Scanpath Images - AP')

scanpathPreds = []
names = []
for i in range(8):
  testData = Image2Data('RGB', test[i], path, imageFolder='Scanpath Images - AP') 
  testX, testy  = testData.X, testData.y
  subjectNames = testData.names
  sumPreds = np.zeros((testy.shape[0],1))
  for r in range(10):
    model = keras.models.load_model(f'{path}/AP Models/model_scanpath_images_ap_run_{r}_{i}')
    preds = (model.predict(testX)).reshape(testy.shape[0],1)  
    sumPreds += preds
  meanPreds = sumPreds/10
  scanpathPreds.append(meanPreds)
  names.extend(subjectNames)

temporalPreds = []
for i in range(8):
  testData = Image2Data('RGB', test[i], path, imageFolder='Temporal Images - AP') 
  testX, testy  = testData.X, testData.y
  sumPreds = np.zeros((testy.shape[0],1))
  for r in range(10):
    model = keras.models.load_model(f'{path}/AP Models/model_temporal_images_ap_run_{r}_{i}')
    preds = (model.predict(testX)).reshape(testy.shape[0],1)  
    sumPreds += preds
  meanPreds = sumPreds/10
  temporalPreds.append(meanPreds)

hybridPreds = []
for i in range(8):
  hybridPreds.extend(scanpathPreds[i]*HybridDf.Scanpath_Model_Weight[i]+temporalPreds[i]*HybridDf.Temporal_Model_Weight[i])

In [ ]:
apcontrolDf = pd.DataFrame.from_dict({'names':names,'preds':hybridPreds})
apcontrolDf['Group'] = apcontrolDf['names'].apply(lambda x: 'Control' if x[-4:] == 'cont' else 'AP')
apcontrolDf.preds = apcontrolDf.preds.apply(lambda x: x[0])

In [ ]:
import plotly.figure_factory as ff
import numpy as np 

hist_data = [apcontrolDf.preds]
group_labels = ['distplot'] # name of the dataset

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False,show_rug=False, curve_type ='kde',colors=['grey'])

# Add title
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',legend=dict(font_size=20),width=1000)
fig.update_yaxes(title='Probability Density',showticklabels=False,title_font_size=25,tickfont=dict(size=25))
fig.update_xaxes(title='Predictions',title_font_size=25,tickfont=dict(size=25))

fig.show()

In [ ]:
print(apcontrolDf['preds'].quantile(q=0.1))
print(apcontrolDf['preds'].quantile(q=0.9))

0.1617634642124176
0.6715312594175339


In [ ]:
aps = apcontrolDf[apcontrolDf['Group']=='AP']
aps['names'] = aps['names'].map({'agday1':'RIOT-4', 'bp_day1':'B-AT2', 'tswday1':'B-AT1', 'rc_p':'R-AT3', 'cp_day1':'B-IOT2', 'am_p':'R-AT2', 'rgsep18':'B-ATOT2', 'ds':'L-IOT2'})
aps_sorted = aps.groupby(['Group','names']).median().reset_index().sort_values('preds').names.to_list()

# Create distplot with curve_type set to 'normal'
fig = px.box(aps, 'names', 'preds',category_orders={'names':aps_sorted})

# Add title
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',legend=dict(font_size=20))
fig.update_traces(marker_color='gray')

fig.update_yaxes(title='Distribution of Predictions',title_font_size=30,tickfont=dict(size=30))
fig.update_xaxes(title_font_size=30,tickfont=dict(size=30))

fig.show()

# Visualizing DP and Control Predictions

In [ ]:
hybridDf = pd.read_csv(path+'/DP_Hybrid_Results.csv')
_ , _, test = getTrainTest(path,positiveClass='DP', valSet=True,imageFolder='Scanpath Images - DP')

scanpathPreds = []
names = []
for i in range(10):
  testData = Image2Data('RGB', test[i], path, imageFolder='Scanpath Images - DP') 
  testX, testy  = testData.X, testData.y
  subjectNames = testData.names
  sumPreds = np.zeros((testy.shape[0],1))
  for r in range(10):
    model = keras.models.load_model(f'{path}/DP Models/model_scanpath_images_dp_run_{r}_{i}')
    preds = (model.predict(testX)).reshape(testy.shape[0],1)  
    sumPreds += preds
  meanPreds = sumPreds/10
  scanpathPreds.append(meanPreds)
  names.extend(subjectNames)

temporalPreds = []
for i in range(10):
  testData = Image2Data('RGB', test[i], path, imageFolder='Temporal Images - DP') 
  testX, testy  = testData.X, testData.y
  sumPreds = np.zeros((testy.shape[0],1))
  for r in range(10):
    model = keras.models.load_model(f'{path}/DP Models/model_temporal_images_dp_run_{r}_{i}')
    preds = (model.predict(testX)).reshape(testy.shape[0],1)  
    sumPreds += preds
  meanPreds = sumPreds/10
  temporalPreds.append(meanPreds)

hybridPreds = []
for i in range(10):
  hybridPreds.extend(scanpathPreds[i]*hybridDf.Scanpath_Model_Weight[i]+temporalPreds[i]*hybridDf.Temporal_Model_Weight[i])

In [ ]:
dpcontrolDf = pd.DataFrame.from_dict({'names':names,'preds':hybridPreds})
dpcontrolDf['Group'] = dpcontrolDf['names'].apply(lambda x: 'Control' if x[-4:] == 'cont' else 'DP')
dpcontrolDf.preds = dpcontrolDf['preds'].apply(lambda x: x[0])

In [ ]:
import plotly.figure_factory as ff
import numpy as np 

hist_data = [dpcontrolDf.preds]
group_labels = ['distplot'] # name of the dataset

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False,show_rug=False, curve_type ='kde',colors=['grey'])

# Add title
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',legend=dict(font_size=20),width=1000)
fig.update_yaxes(title='Probability Density',showticklabels=False,title_font_size=25,tickfont=dict(size=25))
fig.update_xaxes(title='Predictions',title_font_size=25,tickfont=dict(size=25))

fig.show()

In [ ]:
print(dpcontrolDf['preds'].quantile(q=0.1))
print(dpcontrolDf['preds'].quantile(q=0.9))

0.3882470905780792
0.6245566195434517


# Visualizing DP predictions Using AP Hybrid Model

In [ ]:
HybridDf = pd.read_csv(path+'/AP_Hybrid_Results.csv')
subjects = os.listdir(path+'/Scanpath Images - DP - 4')
DPs = [i for i in subjects if i[-4:] != 'cont']

meanPreds = []
names = []

for subject in range(10):

    testData = Image2Data('RGB', [DPs[subject]], path, imageFolder='Scanpath Images - DP - 4') 
    testX, testy  = testData.X, testData.y
    
    scanpathPreds = []
    for i in range(8):
      sumPreds = np.zeros((testy.shape[0],1))
      for r in range(10):
        model = keras.models.load_model(f'{path}/AP Models/model_scanpath_images_ap_run_{r}_{i}')
        preds = (model.predict(testX)).reshape(testy.shape[0],1)  
        sumPreds += preds
      scanpathPreds.append(sumPreds/10)


    testData = Image2Data('RGB', [DPs[subject]], path, imageFolder='Temporal Images - DP - 4') 
    testX, testy  = testData.X, testData.y
    temporalPreds = []
    for i in range(8):
      sumPreds = np.zeros((testy.shape[0],1))
      for r in range(10):
        model = keras.models.load_model(f'{path}/AP Models/model_temporal_images_ap_run_{r}_{i}')
        preds = (model.predict(testX)).reshape(testy.shape[0],1)  
        sumPreds += preds
      temporalPreds.append(sumPreds/10)

    hybridPreds = []
    for i in range(8):
      hybridPreds.append(scanpathPreds[i]*HybridDf.Scanpath_Model_Weight[i]+temporalPreds[i]*HybridDf.Temporal_Model_Weight[i])
    
    meanHybridPreds = sum(hybridPreds)/len(hybridPreds)
    meanPreds.extend(meanHybridPreds)
    names.extend([DPs[subject] for i in range(len(testX))])

In [ ]:
dps = pd.DataFrame.from_dict({'names':names,'preds':meanPreds})
dps['Group'] = 'DP'
dps.preds = dps['preds'].apply(lambda x: x[0])

In [ ]:
import plotly.figure_factory as ff
import numpy as np 
import plotly.express as px


dps_sorted = dps.groupby(['Group','names']).median().reset_index().sort_values('preds').names.to_list()
# Create distplot with curve_type set to 'normal'
fig = px.box(dps, 'names', 'preds',category_orders={'names':dps_sorted})

# Add title
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',legend=dict(font_size=20),width=2500)
fig.update_traces(marker_color='gray')
fig.update_yaxes(title='Distribution of Predictions',title_font_size=30,tickfont=dict(size=30))
fig.update_xaxes(title_font_size=30,tickfont=dict(size=30))

fig.show()

#Graphing all Preds 

In [ ]:
allData = pd.concat([apcontrolDf,dps])

In [ ]:
import plotly.figure_factory as ff
import numpy as np 

hist_data = [allData['preds'][allData['Group']=='AP'], allData['preds'][allData['Group']=='Control'], allData['preds'][allData['Group']=='DP']]

group_labels = ['APs', 'Controls', 'DPs']
colors = ['dimgray', 'black','lightgray']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors,show_rug=False)

# Add title
fig.update_layout(paper_bgcolor='white',plot_bgcolor='white',legend=dict(font_size=30))
fig.update_yaxes(title='Frequency Distribution',title_font_size=35,tickfont=dict(size=30))
fig.update_xaxes(title='Predictions',title_font_size=35,tickfont=dict(size=35))

fig.show()

In [ ]:
allData.groupby('Group').mean()

,preds
Group,
AP,0.491363
Control,0.319128
DP,0.370977
